In [ ]:
%matplotlib inline
%matplotlib notebook
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from random import randint

In [ ]:
sequence_path = "sequence\\"
background_img = "background.png"

# path = base path
# folders = folders within path that contain images
# Returns a list of strings of the relative path names for every image in path/ within every folder in folders
def get_files(path):
    file_list = list()
    for (dir_path, dir_names, file_names) in os.walk(path):
        file_list += [os.path.join(dir_path, file) for file in file_names]
    return file_list


In [ ]:
def get_blob_bounds(img):
    bounding_rects = np.empty((0,4))
    im, contours, hierarchy = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    if (len(img.shape) < 3):
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        bounding_rects = np.vstack((bounding_rects, np.array((x,y,w,h))))
    return bounding_rects, img

# r is the "reach" -> the distance each bounding box tries to merge with other bounding boxes (in each direction)
def merge_bounding_rects(rects,r):
    for i in range(rects.shape[0]):
        if (rects[i] is None):
            continue
        for j in range(rects.shape[0]):
            if (rects[j] is None):
                continue
            i_x, i_y, i_w, i_h = rects[i]
            j_x, j_y, j_w, j_h = rects[j]
            # Ignore equal rects
            if (i_x, i_y, i_w, i_h) == (j_x, j_y, j_w, j_h):
                continue
            # Check if rectangle i (increased by ratio) overlaps with rectangle j
            x1 = i_x - r
            y1 = i_y + i_h + r
            x2 = i_x + i_w + r
            y2 = i_y - r
            
            x3 = j_x
            y3 = j_y + j_h
            x4 = j_x + j_w
            y4 = j_y
            
            # Check if current bounding box plus reach overlaps with any other bounding boxes (meaning they are within reach)
            if ((x1 < x4) and (x3 < x2) and (y1 > y4) and (y3 > y2)):
                # Calculate new size of bounding box
                if (i_x > j_x):
                    # Special case
                    if ((i_x + i_w) < (j_x + j_w)):
                        new_w = j_w
                    else:
                        new_w = i_x - j_x + i_w
                else:
                    if ((i_x + i_w) > (j_x + j_w)):
                        new_w = i_w
                    else:
                        new_w = j_x - i_x + j_w
                if (i_y > j_y):
                    # Special case
                    if ((i_y + i_h) < (j_y + j_h)):
                        new_h = j_h
                    else:
                        new_h = i_y - j_y + i_h
                else:
                    if ((i_y + i_h) > (j_y + j_h)):
                        new_h = i_h
                    else:
                        new_h = j_y - i_y + j_h
                    
                new_x = min(i_x,j_x)
                new_y = min(i_y,j_y)
                rects[i] = np.array((new_x, new_y, new_w, new_h))
                # Remove other bounding box (current bounding box essentially "consumes" it)
                rects[j] = None
    return rects


def enlarge_rect(rects, percent_x, percent_y, cap_x, cap_y, base_x, base_y):
    # Calculate increase in dimensions based on the width/height of the box
    x_increase = percent_x*rects[:,2]
    y_increase = percent_y*rects[:,3]
    # Cap the increase so it doesn't make large merged bounding boxes too much bigger than necessary
    x_increase[x_increase > cap_x] = cap_x
    y_increase[y_increase > cap_y] = cap_y
    x_increase[x_increase < base_x] = base_x
    y_increase[y_increase < base_y] = base_y
    
    # Increase bounding box
    rects[:,0] = rects[:,0] - x_increase
    rects[:,1] = rects[:,1] - y_increase
    rects[:,2] = rects[:,2] + 2*x_increase
    rects[:,3] = rects[:,3] + 2*y_increase
    
    return rects


# Setup plot
plt.close('all')
fig = plt.figure()
fig.set_figwidth(20)
fig.set_figheight(5)
fig.tight_layout = True
fig.subplotpars.left=0
fig.subplotpars.top=1
fig.subplotpars.bottom=0
fig.subplotpars.wspace=0.1
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)
plt.ion()
fig.show()
fig.canvas.draw()

k_open = np.ones((3,3),np.uint8)
k_close = np.ones((15,15), np.uint8)
k_erode = np.ones((5,5),np.uint8)
k_dilate = np.ones((5,5),np.uint8)

images = get_files(sequence_path)
bg_img = cv2.imread(background_img)
bg_img_gray = cv2.imread(background_img, 0)
for n, i in enumerate(images[150:]):
    ax1.clear()
    ax2.clear()
    ax3.clear()
    img = cv2.imread(i)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    subtract_1 = cv2.absdiff(bg_img, img)
    subtract = np.max(subtract_1, axis=2)
    
    ret, thresholded = cv2.threshold(subtract, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    morphed = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, k_open)
    morphed = cv2.morphologyEx(morphed, cv2.MORPH_ERODE, k_erode)
    morphed = cv2.morphologyEx(morphed, cv2.MORPH_DILATE, k_dilate)
    morphed = cv2.morphologyEx(morphed, cv2.MORPH_CLOSE, k_close)

    r, img2 = get_blob_bounds(morphed)
    
    r = merge_bounding_rects(r,50)
    r = r[~np.isnan(r).any(axis=1)]

    r = enlarge_rect(r, 0.4, 0.2, 30, 50, 30, 30)
    
    for i in r.astype(np.int32):
        x,y,w,h = i
        cv2.rectangle(img2,(x,y),(x+w,y+h),(255,0,0),3)
        img2[max(y,0):y+h,max(x,0):x+w,:] = img[max(y,0):y+h,max(x,0):x+w,:]

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    ax1.imshow(img)
    ax2.imshow(img2)
    ax3.imshow(morphed,cmap='gray')
    title = str('Frame = ' + str(n))
    plt.title(title)
#     plt.pause(.1)
    fig.canvas.draw()